In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directorych
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses,regularizers,optimizers

In [3]:
(x_train,y_train),(x_test,y_test)=datasets.mnist.load_data()
x_train, y_train = x_train[:1000], y_train[:1000]
x_test, y_test = x_test[:200],y_test[:200]  

print(x_train.shape)  
print(x_test.shape) 

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(1000, 28, 28)
(200, 28, 28)


In [4]:
x_train = tf.pad(x_train, [[0, 0], [99, 100], [99, 100]]) / 255
x_test = tf.pad(x_test, [[0, 0], [99, 100], [99, 100]]) / 255
x_train.shape

TensorShape([1000, 227, 227])

In [5]:
# Add a single channel (grayscale)
x_train = tf.expand_dims(x_train, axis=-1)  # (batch, 227, 227, 1)
x_test = tf.expand_dims(x_test, axis=-1)

# Convert grayscale to RGB (repeat the single channel 3 times)
x_train = tf.repeat(x_train, 3, axis=-1)  # (batch, 227, 227, 3)
x_test = tf.repeat(x_test, 3, axis=-1)
x_train.shape

TensorShape([1000, 227, 227, 3])

In [6]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),  # Random left-right flip
    layers.RandomRotation(0.1),       # Small random rotation
    layers.RandomZoom(0.1),           # Random zoom-in or zoom-out
    layers.RandomBrightness(0.1),     # Adjust brightness randomly
    layers.RandomContrast(0.1)        # Adjust contrast randomly
])

# Apply augmentation to training images
x_train_augmented = data_augmentation(x_train)

In [7]:
model=models.Sequential()
l2_lambda = 0.0005  
model.add(layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', 
                        activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), 
                        input_shape=(227,227,3)))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same', 
                        activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', 
                        activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))

model.add(layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', 
                        activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))

model.add(layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', 
                        activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))

model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(4096,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,activation='softmax'))

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 55, 55, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 27, 27, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 27, 27, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 13, 13, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 13, 13, 384)         │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 13, 13, 384)         │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 13, 13, 256)         │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │      37,752,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │          40,970 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 58,322,314 (222.48 MB)

 Trainable params: 58,322,314 (222.48 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Compile Model
model.compile(optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss=losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [9]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.1336 - loss: 2.8719 - val_accuracy: 0.1200 - val_loss: 2.8585
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.1196 - loss: 2.8632 - val_accuracy: 0.1450 - val_loss: 2.8483
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.1385 - loss: 2.8449 - val_accuracy: 0.1200 - val_loss: 2.8318
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.1687 - loss: 2.8191 - val_accuracy: 0.3000 - val_loss: 2.7305
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.2595 - loss: 2.6611 - val_accuracy: 0.2450 - val_loss: 2.5148
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.1594 - loss: 2.8053 - val_accuracy: 0.3300 - val_loss: 2.3443
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.3858 - loss: 2.1758 - val_accuracy: 0.5900 - val_loss: 1.8283
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.5001 - loss: 1.8816 - val_accuracy: 0.6750 - val_loss: